In [1]:
import os
import random
import itertools
import re

# 基本的なライブラリ
import numpy as np
import pandas as pd
import scipy as sp
from scipy import stats

# 描画ライブラリ
import matplotlib.pyplot as plt
import seaborn as sns
from seaborn_analyzer import CustomPairPlot
import graphviz
import pydotplus
from IPython.display import Image
from IPython.display import HTML
from six import StringIO
from ipywidgets import interact, FloatSlider

# 前処理
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from imblearn.over_sampling import SMOTE
from sklearn.feature_selection import VarianceThreshold

# 補完
from sklearn.experimental import (
    enable_iterative_imputer,
)  # IterativeImputerをimportするために必要
from sklearn.impute import SimpleImputer, IterativeImputer, KNNImputer

# エンコード
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, OrdinalEncoder

# データセット分割
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

# 特徴量選択
from sklearn.feature_selection import (
    GenericUnivariateSelect,
    f_classif,
    mutual_info_classif,
    chi2,
)
from boruta import BorutaPy

# 学習中
from tqdm import tqdm
from sklearn.model_selection import learning_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

import warnings


# config python file
import config

SEED = config.SEED


from functions import *

fix_seed(SEED)


# 最大表示列数の指定（ここでは50列を指定）N
pd.set_option("display.max_columns", 50)
pd.set_option("display.max_rows", 50)

%matplotlib inline

/Users/masanari/Library/CloudStorage/OneDrive-東京理科大学/projects/breast_cancer_analytics/notebooks/functions.py:278: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  y: pd.Series(),


# 目的
遺伝子学的分類に基づいた、予後の2値分類を実施する。  
分類はCLAUDIN_SUBTYPEに基づいて実施。  
予後は5年、10年、15年の3つの年次に分けている。Trueで死亡であることに注意すること。

# データ読み込み
読み込み元：
    config.INTERIM_PICKLE_PREPROCESSED_PROGNOSIS_CROSS_DIR + "/claudin_subtype_chi2"

サブタイプ毎のデータを使用

データの種類が多いので、辞書型で表現する  
```        
+/X_dict
    +/train
        +/X5
            +.X_chi2
            +.X_subtypes
        +/X10
            +.X_chi2
            +.X_subtypes
        +/X15
            +.X_chi2
            +.X_subtypes
    +/test
        +/X5
            +.X_chi2
            +.X_subtypes
        +/X10
            +.X_chi2
            +.X_subtypes
        +/X15
            +.X_chi2
            +.X_subtypes

+/y_dict
    +/train
        +.y5
            +.y_subtypes
        +.y10
            +.y_subtypes
        +.y15
            +.y_subtypes
    +/test
        +.y5
            +.y_subtypes
        +.y10
            +.y_subtypes
        +.y15
            +.y_subtypes
        
        
```

In [2]:
# データが複数種類あるので、辞書に保存する
X_dict = {}
y_dict = {}

# 前処理済みのデータの保存ディレクトリ（テーブル・遺伝子混合）
dir_path = config.INTERIM_PICKLE_PREPROCESSED_PROGNOSIS_CROSS_DIR
# データのカテゴリごとのループ
for dir_cat in [
    d for d in os.listdir(dir_path) if os.path.isdir(os.path.join(dir_path, d))
]:
    if not dir_cat in X_dict:
        X_dict[dir_cat] = dict()  # ネストの辞書なので、キーに対応する辞書がなければ生成する
        y_dict[dir_cat] = dict()
    # train、testのステップ毎のループ
    for dir_step in [
        d for d in os.listdir(dir_path + "/" + dir_cat) if not d.startswith(".")
    ]:
        if not dir_step in X_dict[dir_cat]:
            X_dict[dir_cat][dir_step] = dict()
            y_dict[dir_cat][dir_step] = dict()
        for f_name in tqdm(os.listdir(dir_path + "/" + dir_cat + "/" + dir_step)):
            if f_name[0] == "X":
                X_dict[dir_cat][dir_step][f_name] = pd.read_pickle(
                    dir_path + "/" + dir_cat + "/" + dir_step + "/" + f_name
                )
            if f_name[0] == "y":
                y_dict[dir_cat][dir_step][f_name] = pd.read_pickle(
                    dir_path + "/" + dir_cat + "/" + dir_step + "/" + f_name
                )
        X_dict[dir_cat][dir_step] = dict(sorted(X_dict[dir_cat][dir_step].items()))
        y_dict[dir_cat][dir_step] = dict(sorted(y_dict[dir_cat][dir_step].items()))

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 36/36 [00:00<00:00, 4616.59it/s]


# モデルのトレーニング

## データ全体のベースライン・学習

In [3]:
for (kx, vx), (ky, vy) in zip(
    X_dict["chi2"]["train"].items(),
    y_dict["chi2"]["train"].items(),
):  # X, yを同時にループ
    assert kx[1:3] == ky[1:3]  # カテゴリが同じかの確認
    if accuracy_score(vy, np.zeros(len(vy))) >= 0.5:
        print('"0">"1"', ky, round(accuracy_score(vy, np.zeros(len(vy))), 3))
    else:
        print('"1">"0"', ky, round(accuracy_score(vy, np.ones(len(vy))), 3))

"0">"1" y05.pickle 0.812
"0">"1" y10.pickle 0.636
"1">"0" y15.pickle 0.533


In [4]:
# 各予後毎の、ベースモデルの学習
for (kx, vx), (ky, vy) in zip(
    X_dict["chi2"]["train"].items(),
    y_dict["chi2"]["train"].items(),
):
    print("-----" * 10)
    assert kx[1:3] == ky[1:3]
    print(kx)
    print("使用特徴量", vx.columns)
    print("学習サンプルサイズ：", vx.shape)
    print(vy.value_counts())
    display(compare_bcms(vx, vy))

--------------------------------------------------
X05.pickle
使用特徴量 Index(['FGD3', 'LRIG1', 'ANXA9', 'PDZK1', 'CCNE1', 'STAC2', 'PTTG1', 'AIF1L',
       'KIAA1467', 'SEC14L2',
       ...
       'SYBU', 'FLNB', 'TTYH1', 'PI15', 'RBBP8', 'ARSG', 'BTG2', 'MCM10',
       'DIO1', 'SLC7A8'],
      dtype='object', length=355)
学習サンプルサイズ： (1306, 355)
0    1060
1     246
Name: OS_05years, dtype: int64


11it [00:41,  3.74s/it]


,acc_train,acc_test,f1_train,f1_test
classifier,,,,
Naive Bayes,0.705887,0.705937,0.448629,0.442233
Linear SVM,0.951420,0.728949,0.861150,0.319061
Polynomial SVM,1.000000,0.727405,1.000000,0.307344
Logistic Regression,0.927769,0.736577,0.786245,0.306263
AdaBoost,0.896800,0.774874,0.680265,0.266679
Nearest Neighbors,0.834525,0.772584,0.430911,0.231033
Decision Tree,0.881658,0.740446,0.643464,0.229440
Random Forest,0.848137,0.807804,0.322525,0.008333
Quadratic Discriminant Analysis,0.969117,0.811650,0.910602,0.000000


--------------------------------------------------
X10.pickle
使用特徴量 Index(['FGD3', 'LRIG1', 'ANXA9', 'PDZK1', 'CCNE1', 'STAC2', 'PTTG1', 'AIF1L',
       'KIAA1467', 'SEC14L2',
       ...
       'FLNB', 'TTYH1', 'PI15', 'RBBP8', 'HIST1H2BG', 'ARSG', 'BTG2', 'MCM10',
       'DIO1', 'SLC7A8'],
      dtype='object', length=384)
学習サンプルサイズ： (1048, 384)
0    667
1    381
Name: OS_10years, dtype: int64


11it [00:38,  3.49s/it]


,acc_train,acc_test,f1_train,f1_test
classifier,,,,
Naive Bayes,0.656701,0.649817,0.562034,0.548122
AdaBoost,0.856551,0.649780,0.794919,0.494966
Logistic Regression,0.911048,0.631740,0.875205,0.488242
Polynomial SVM,1.000000,0.626987,1.000000,0.486947
Linear SVM,0.944763,0.620311,0.922982,0.477744
Nearest Neighbors,0.748623,0.627912,0.640221,0.469255
Decision Tree,0.823473,0.631612,0.739205,0.446622
RBF SVM,0.784351,0.674643,0.646573,0.443723
Random Forest,0.876060,0.668855,0.801180,0.410713


--------------------------------------------------
X15.pickle
使用特徴量 Index(['FGD3', 'LRIG1', 'ANXA9', 'PDZK1', 'CCNE1', 'STAC2', 'PTTG1', 'AIF1L',
       'KIAA1467', 'SEC14L2',
       ...
       'FLNB', 'TTYH1', 'PI15', 'RBBP8', 'HIST1H2BG', 'ARSG', 'BTG2', 'MCM10',
       'DIO1', 'SLC7A8'],
      dtype='object', length=395)
学習サンプルサイズ： (811, 395)
1    432
0    379
Name: OS_15years, dtype: int64


11it [00:28,  2.59s/it]


,acc_train,acc_test,f1_train,f1_test
classifier,,,,
RBF SVM,0.800384,0.670837,0.823735,0.705152
Sigmoid SVM,0.532677,0.532746,0.695061,0.692588
Random Forest,0.866419,0.646161,0.878101,0.692261
AdaBoost,0.885600,0.627582,0.893471,0.655209
Nearest Neighbors,0.748187,0.624044,0.771799,0.651356
Naive Bayes,0.644472,0.637504,0.647227,0.637349
Logistic Regression,0.962736,0.584463,0.964937,0.604978
Polynomial SVM,1.000000,0.590681,1.000000,0.604849
Linear SVM,0.989725,0.583273,0.990342,0.602943


# Borutaによる特徴量選択

In [5]:
for i in range(5, 16, 5):  # 全部の特徴量で学習
    print("----------" * 10)
    print("index: ", format(str(i).zfill(2)))
    rf1 = RandomForestClassifier(n_jobs=-1, max_depth=5, random_state=SEED)
    rf1.fit(
        X_dict["chi2"]["train"]["X{0}.pickle".format(str(i).zfill(2))],
        y_dict["chi2"]["train"]["y{0}.pickle".format(str(i).zfill(2))],
    )
    rf1_pred = rf1.predict(
        X_dict["chi2"]["train"]["X{0}.pickle".format(str(i).zfill(2))]
    )
    print("train score")
    print(
        show_scores(
            y_dict["chi2"]["train"]["y{0}.pickle".format(str(i).zfill(2))], rf1_pred
        )
    )
    rf1_pred = rf1.predict(
        X_dict["chi2"]["test"]["X{0}.pickle".format(str(i).zfill(2))]
    )
    print("test score")
    print(
        show_scores(
            y_dict["chi2"]["test"]["y{0}.pickle".format(str(i).zfill(2))], rf1_pred
        )
    )

    # RandomForestRegressorでBorutaを実行
    rf = RandomForestClassifier(n_jobs=-1, max_depth=5)
    feat_selector = BorutaPy(rf, n_estimators="auto", verbose=0, random_state=SEED)
    feat_selector.fit(
        X_dict["chi2"]["train"]["X{0}.pickle".format(str(i).zfill(2))].values,
        y_dict["chi2"]["train"]["y{0}.pickle".format(str(i).zfill(2))].values,
    )

    # 選択された特徴量を確認
    selected = feat_selector.support_
    print("選択された特徴量の数:{0}".format(np.sum(selected)))
    print(selected)
    print(
        X_dict["chi2"]["train"]["X{0}.pickle".format(str(i).zfill(2))].columns[selected]
    )

    # 選択した特徴量で学習
    X_selected = X_dict["chi2"]["train"]["X{0}.pickle".format(str(i).zfill(2))][
        X_dict["chi2"]["train"]["X{0}.pickle".format(str(i).zfill(2))].columns[selected]
    ]
    rf2 = RandomForestClassifier(n_jobs=-1, max_depth=5, random_state=SEED)
    rf2.fit(X_selected, y_dict["chi2"]["train"]["y{0}.pickle".format(str(i).zfill(2))])
    rf2_pred = rf2.predict(
        X_dict["chi2"]["train"]["X{0}.pickle".format(str(i).zfill(2))][
            X_selected.columns
        ]
    )
    print(
        show_scores(
            y_dict["chi2"]["train"]["y{0}.pickle".format(str(i).zfill(2))], rf2_pred
        )
    )
    rf2_pred = rf2.predict(
        X_dict["chi2"]["test"]["X{0}.pickle".format(str(i).zfill(2))][
            X_selected.columns
        ]
    )
    print(
        show_scores(
            y_dict["chi2"]["test"]["y{0}.pickle".format(str(i).zfill(2))], rf2_pred
        )
    )

----------------------------------------------------------------------------------------------------
index:  05
train score
accuracy:  0.8522205206738132
precision:  1.0
recall:  0.21544715447154472
f1 score:  0.3545150501672241
None
test score
accuracy:  0.8211009174311926
precision:  0.0
recall:  0.0
f1 score:  0.0
None


/Users/masanari/.anyenv/envs/pyenv/versions/miniforge3-4.10/envs/breast-cancer-analytics/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/masanari/.anyenv/envs/pyenv/versions/miniforge3-4.10/envs/breast-cancer-analytics/lib/python3.8/site-packages/boruta/boruta_py.py:319: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  self.support_ = np.zeros(n_feat, dtype=np.bool)
/Users/masanari/.anyenv/envs/pyenv/versions/miniforge3-4.10/envs/breast

選択された特徴量の数:26
[ True False False False False False False False False False False False
 False False False False False False False False False  True False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False  True  True False False False False False False False  True
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False  True False False False False False False False False False
 False False False False False False False False False False False False
 False False  True False False  True False False False False False False
 False False False False False False False False False False False False
 False False False False False  True False False False False False False
 False False False False False False 

/Users/masanari/.anyenv/envs/pyenv/versions/miniforge3-4.10/envs/breast-cancer-analytics/lib/python3.8/site-packages/boruta/boruta_py.py:319: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  self.support_ = np.zeros(n_feat, dtype=np.bool)
/Users/masanari/.anyenv/envs/pyenv/versions/miniforge3-4.10/envs/breast-cancer-analytics/lib/python3.8/site-packages/boruta/boruta_py.py:321: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and gui

選択された特徴量の数:41
[ True False False False False False  True False False False False False
 False False  True False False False False False False  True False False
  True False False False False False False False  True False False False
 False False False False False False  True False  True False False False
 False False False False False False False False False False False  True
 False False False False False False False  True False False False False
 False False False False  True False False False False False False False
 False False False False False False False False False False False False
 False False False  True False False False False False False False False
 False False False  True False False False False False False False  True
 False False False False False False False False False False False False
 False False False  True False False False False False False False False
 False False False False False False False False False False False False
 False False False False False  True 

/Users/masanari/.anyenv/envs/pyenv/versions/miniforge3-4.10/envs/breast-cancer-analytics/lib/python3.8/site-packages/boruta/boruta_py.py:319: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  self.support_ = np.zeros(n_feat, dtype=np.bool)
/Users/masanari/.anyenv/envs/pyenv/versions/miniforge3-4.10/envs/breast-cancer-analytics/lib/python3.8/site-packages/boruta/boruta_py.py:321: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and gui

## subtype毎のベースライン・学習

In [6]:
for (kx, vx), (ky, vy) in zip(
    X_dict["claudin_subtype_chi2"]["train"].items(),
    y_dict["claudin_subtype_chi2"]["train"].items(),
):
    assert kx.split("_")[1] == ky.split("_")[1]
    if accuracy_score(vy, np.zeros(len(vy))) >= 0.5:
        print('"0">"1"', ky, round(accuracy_score(vy, np.zeros(len(vy))), 3))
    else:
        print('"1">"0"', ky, round(accuracy_score(vy, np.ones(len(vy))), 3))

"0">"1" y05_Basal.pickle 0.655
"0">"1" y05_Her2.pickle 0.667
"0">"1" y05_LumA.pickle 0.923
"0">"1" y05_LumB.pickle 0.788
"0">"1" y05_Normal.pickle 0.821
"0">"1" y05_claudin-low.pickle 0.812
"0">"1" y10_Basal.pickle 0.557
"1">"0" y10_Her2.pickle 0.558
"0">"1" y10_LumA.pickle 0.783
"0">"1" y10_LumB.pickle 0.54
"0">"1" y10_Normal.pickle 0.646
"0">"1" y10_claudin-low.pickle 0.692
"1">"0" y15_Basal.pickle 0.577
"1">"0" y15_Her2.pickle 0.684
"0">"1" y15_LumA.pickle 0.626
"1">"0" y15_LumB.pickle 0.675
"1">"0" y15_Normal.pickle 0.571
"0">"1" y15_claudin-low.pickle 0.575


In [7]:
for (kx, vx), (ky, vy) in zip(
    X_dict["claudin_subtype_chi2"]["train"].items(),
    y_dict["claudin_subtype_chi2"]["train"].items(),
):
    print("-----" * 10)
    assert kx.split("_")[1] == ky.split("_")[1]
    print(kx)
    print("使用特徴量", vx.columns)
    print("学習サンプルサイズ：", vx.shape)
    print(vy.value_counts())
    display(compare_bcms(vx, vy))

--------------------------------------------------
X05_Basal.pickle
使用特徴量 Index(['CXCL13', 'CSN3', 'SCGB2A2', 'IGHG1'], dtype='object')
学習サンプルサイズ： (148, 4)
0    97
1    51
Name: OS_05years, dtype: int64


11it [00:01,  9.43it/s]


,acc_train,acc_test,f1_train,f1_test
classifier,,,,
Naive Bayes,0.722209,0.694762,0.626209,0.574322
Quadratic Discriminant Analysis,0.728212,0.681905,0.634223,0.563464
Linear SVM,0.727488,0.681905,0.566543,0.506810
Logistic Regression,0.716205,0.701905,0.537419,0.503080
Polynomial SVM,0.740989,0.632381,0.586607,0.476721
AdaBoost,0.939165,0.642381,0.908858,0.446189
Random Forest,0.768028,0.627143,0.612353,0.388636
Decision Tree,0.757491,0.606190,0.623461,0.370668
Nearest Neighbors,0.758983,0.634286,0.610418,0.343997


--------------------------------------------------
X05_Her2.pickle
使用特徴量 Index(['U79293', 'KRT81', 'GFRA1', 'ESR1', 'SCUBE2', 'CLCA2'], dtype='object')
学習サンプルサイズ： (153, 6)
0    102
1     51
Name: OS_05years, dtype: int64


11it [00:01,  7.95it/s]


...

11it [00:00, 12.11it/s]


,acc_train,acc_test,f1_train,f1_test
classifier,,,,
AdaBoost,1.000000,0.880000,1.000000,0.879762
Logistic Regression,0.863098,0.806667,0.880411,0.824206
Polynomial SVM,0.996000,0.820000,0.996487,0.809365
Linear SVM,0.876980,0.773333,0.888745,0.783175
Nearest Neighbors,0.865020,0.776667,0.868783,0.771984
Naive Bayes,0.847216,0.786667,0.861549,0.768254
RBF SVM,0.908745,0.730000,0.916718,0.744206
Quadratic Discriminant Analysis,0.910784,0.710000,0.918372,0.735873
Random Forest,0.722314,0.623333,0.800582,0.726825


--------------------------------------------------
X15_claudin-low.pickle
使用特徴量 Index(['ROPN1B', 'PROM1', 'SOX10', 'ROPN1', 'SNAR-A3', 'S100A9', 'SFRP1',
       'C4orf7', 'ELF5', 'GABRP'],
      dtype='object')
学習サンプルサイズ： (80, 10)
0    46
1    34
Name: OS_15years, dtype: int64


11it [00:00, 11.28it/s]


,acc_train,acc_test,f1_train,f1_test
classifier,,,,
Naive Bayes,0.722222,0.7000,0.706563,0.585303
Random Forest,0.734722,0.6625,0.708905,0.547381
AdaBoost,1.000000,0.6625,1.000000,0.531984
Logistic Regression,0.761111,0.6625,0.704146,0.519365
Quadratic Discriminant Analysis,0.829167,0.6375,0.790251,0.498095
Polynomial SVM,0.916667,0.6250,0.896726,0.492857
Linear SVM,0.755556,0.6250,0.681739,0.475952
Nearest Neighbors,0.833333,0.6750,0.791551,0.462857
RBF SVM,0.823611,0.6250,0.782521,0.457143
